# Informe Tarea 3
## Inteligencia Artifical 2018-2
## Thomas Pryce Jones

<a id='Menu'></a>
### Menú Principal
A continuación se describirá los pasos a seguir para desarrollar la tarea:
+ [Desarrollo](#Development)
    + [Pre-Procesamiento](#Pre_Proces)
    + [Crear Espacio de Características](#Feature_Space)
    + [Generar Vocabularios](#Vocabulary)
    + [Traspasar Vocabularios a notación One Hot](#OneHot)
    + [Modelo SVM](#ModelsSVM)
    + [Modelo NN](#ModelsNN)     
    + [bAbI-Tasks](#bAbI-Tasks)
        + [#1 Basic factoid QA with single supporting fact](#bAbI-1)
        + [#2 Factoid QA with two supporting facts](#bAbI-2)
        + [#3 Factoid QA with three supporting facts](#bAbI-3)
        + [#4 Two argument relations: subject vs. object](#bAbI-4)
        + [#5 Three argument relations](#bAbI-5)
        + [#6 Yes/No questions](#bAbI-6)
        + [#7 Counting](#bAbI-7)
        + [#8 Lists/Sets](#bAbI-8)
        + [#9 Simple Negation](#bAbI-9)
        + [#10 Indefinite Knowledge](#bAbI-10)
        + [#11 Basic coreference](#bAbI-11)
        + [#12 Conjunction](#bAbI-12)
        + [#13 Compound coreference](#bAbI-13)
        + [#14 Time manipulation](#bAbI-14)
        + [#15 Basic deduction](#bAbI-15)
        + [#16 Basic induction](#bAbI-16)
        + [#17 Positional reasoning](#bAbI-17)
        + [#18 Reasoning about size](#bAbI-18)
        + [#19 Path finding](#bAbI-19)
        + [#20 Reasoning about agent's motivation](#bAbI-20)
        + [All Together](#bAbI-ALL)
+ [Conclusión](#Conclusion)

<a id='Pre_Proces'></a>
## Pre-procesamiento

[Volver al Menú](#Menu)


En esta etapa realizaremos el proceso de separación de la información en tuplas de la siguiente manera
    (story, question, answer). De tal forma que nos quedará una tupla con cada una de la información relevante. Además, realizaremos un proceso de tokenización de tal forma de dejar cada una de las oraciones como una lista de strings. Eliminaremos los '.' y llevaremos a minúscula cada linea. Opcionalmente se podrá realizar un proceso de eliminar las stopwords y una aplicación de stemming de cada línea.

In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from IPython.display import display
from pprint import pprint
nltk.download('punkt')
nltk.download('stopwords')

DATASET = [ ["qa1_single-supporting-fact",3],
            ["qa2_two-supporting-facts",3],
            ["qa3_three-supporting-facts",3],
            ["qa4_two-arg-relations",3],
            ["qa5_three-arg-relations",3],
            ["qa6_yes-no-questions",0],
            ["qa7_counting",3],
            ["qa8_lists-sets",3],
            ["qa9_simple-negation",0],
            ["qa10_indefinite-knowledge",0],
            ["qa11_basic-coreference",3],
            ["qa12_conjunction",3],
            ["qa13_compound-coreference",3],
            ["qa14_time-reasoning",3],
            ["qa15_basic-deduction",3],
            ["qa16_basic-induction",3],
            ["qa17_positional-reasoning",0],
            ["qa18_size-reasoning",0],
            ["qa19_path-finding",0],
            ["qa20_agents-motivations",3],
            ["all",2]
          ]

def separate_story_question(data_path, optional = 0):
    '''
        optional:   0 = Sin uso de funciones opcional
                    1 = Usar StopWords
                    2 = Usar Stemming
                    3 = Usar ambas
    '''
    story = []
    question = []
    answer = []
    aux_story = []

    with open(data_path, 'r') as data_file:
        for line in data_file:
            token = word_tokenize(line.lower())
            if (optional == 1 or optional == 3):
                token = filter_stopwords(token)
            if (optional == 2 or optional == 3):
                token = stemming(token)
            if token[0] == '1' in line:
                aux_story = []
            if '?' not in line:
                token = [x for x in filter(lambda a: a != '.', token)]
                # aux_story.append((token[0], token[1:]))
                aux_story.append(token[1:])
            else:
                last_word_index = token.index('?') + 1
                # answer.append((token[last_word_index], token[last_word_index + 1]))
                answer.append(token[last_word_index])
                # question.append((token[0], token[1:last_word_index]))
                question.append(token[1:last_word_index])
                stories = aux_story.copy()
                aux_story.append(token[1:last_word_index])
                story.append(stories)
    return (story, question, answer)


def filter_stopwords(words):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = [w for w in words if w not in stopWords]
    return wordsFiltered

def stemming(words):
    steammer = PorterStemmer()
    st = steammer.stem
    steamWords = [st(word) for word in words]
    return steamWords

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thomasprycejones/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomasprycejones/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def max_story_lenght(stories):
    len_stories = [len(x) for x in stories]
    el_set = set(len_stories)
    return max(len_stories)

def fill_with_fill_token(stories, max_story_number, FILL_TOKEN):
    for story in stories:
        for num in range(max_story_number - len(story)):
            tupla = ([FILL_TOKEN])
            story.append((tupla))
    return stories

def create_register(result):
    registers = []
    for num in range(len(result[0])):
        registers.append([result[0][num], result[1][num], result[2][num]])
    return registers

<a id='Vocabulary'></a>
## Generar Vocabularios

[Volver al Menú](#Menu)

In [3]:
def generate_vocabulary(registers_train, registers_test):
    vocabulary_x = set()
    vocabulary_y = set()
    stories = set()
    for register in registers_train:
        for sentence in register[0]:
            stories = stories.union({word for word in sentence})
        questions = {word for word in register[1]}
        vocabulary_x = vocabulary_x.union(stories)
        vocabulary_x = vocabulary_x.union(questions)
        vocabulary_y = vocabulary_y.union(set([register[2]]))
    for register in registers_test:
        for sentence in register[0]:
            stories = stories.union({word for word in sentence})
        questions = {word for word in register[1]}
        vocabulary_x = vocabulary_x.union(stories)
        vocabulary_x = vocabulary_x.union(questions)
        vocabulary_y = vocabulary_y.union(set([register[2]]))
    vocabulary_x = {word: num for num, word in enumerate(sorted(vocabulary_x))}
    vocabulary_y = {word: num for num, word in enumerate(sorted(vocabulary_y))}
    return vocabulary_x, vocabulary_y

<a id='OneHot'></a>
## Traspasar Vocabularios a notación One Hot

[Volver al Menú](#Menu)

In [4]:
import numpy as np

def create_one_hot(registers, vocab_x, vocab_y):
    x_one_hot_array = []
    y_one_hot_array = []
    for register in registers:
        aux_one_hot_array = []
        for sentence in register[0]:
            aux_one_hot_array.append(bag_of_words(sentence_to_one_hot(sentence, vocab_x)))
        question_one_hot_array = [bag_of_words(sentence_to_one_hot(register[1], vocab_x))]
        aux_one_hot_array.extend(question_one_hot_array)
        x_one_hot_array.append(aux_one_hot_array)
        y_one_hot_array.append(sentence_to_one_hot([register[2]], vocab_y))
    return x_one_hot_array, y_one_hot_array

def bag_of_words(one_hot_words_array):
    return np.sum(one_hot_words_array, axis=0)

def sentence_to_one_hot(words, vocab):
    one_hot_words = []
    for word in words:
        one_hot_words.append(word_to_one_hot(word, vocab))
    return one_hot_words

def word_to_one_hot(word, vocab):
    max_vocab = len(vocab)
    one_hot = np.zeros(max_vocab)
    one_hot[vocab[word]] = 1
    return one_hot

def concatenate_one_hot_array(one_hot_array):
    one_hot = []
    for hot in one_hot_array:
        one_hot.append(np.concatenate(hot, axis=None))
    return one_hot

def sanitize_one_hot(one_hot_array):
    sanitize_one_hot = []
    for f in one_hot_array:
        sanitize_one_hot.append(f[0])
    return sanitize_one_hot

def answers_array(answers_one_hot):
    y = []
    for i in range(len(answers_one_hot)):
        index = np.nonzero(answers_one_hot[i])[1][0]
        y.append(index)
    return y

def one_hot_to_number(one_hot_array):
    counter = 0
    aux = []
    for i in range(len(one_hot_array)):
        if list(one_hot_array[i]).count(1) > 1:
            counter += 1
            aux.append(list(one_hot_array[i]).index(1))
        elif list(one_hot_array[i]).count(1) < 1:
            aux.append(0)
        else:
            aux.append(list(one_hot_array[i]).index(1))
    return aux

<a id='ModelsSVM'></a>
## Modelo SVM

[Volver al Menú](#Menu)

In [5]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def train_SVC(X, y, Coef=1.0):
    clf = LinearSVC(penalty='l2', loss='squared_hinge', C=Coef)
    clf.fit(X, y)
    return clf

def evaluate_SVC(clf, X, y, train_X, train_y, answer_token_map):
    answer_list = sorted(list(answer_token_map.keys()))
    # TESTING SET
    print("\n----------- C={0} -----------".format(clf.C))
    predicted = clf.predict(X)
    score = accuracy_score(y, predicted)
    report = classification_report(y, predicted, target_names=answer_list)
    print("Testing Set Score: {0}".format(score))
    print("Testing Report")
    print(report)
    # TRAINING SET
    predicted = clf.predict(train_X)
    score = accuracy_score(train_y, predicted)
    report = classification_report(train_y, predicted, target_names=answer_list)
    print("Training Set Score: {0}".format(score))
    print("Training Report")
    print(report)

<a id='ModelsNN'></a>
## Modelo NN

[Volver al Menú](#Menu)

In [6]:
from sklearn.neural_network import MLPClassifier

#https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
#https://towardsdatascience.com/beginners-ask-how-many-hidden-layers-neurons-to-use-in-artificial-neural-networks-51466afa0d3e

def train_NN(X, y, hidden=None, iterations = None):
    feature_space_size = len(X[0])
    answers_vocab_size = len(y[0])
    batches = int(len(X)/5)
    h = int(0.7*feature_space_size+0.3*answers_vocab_size)
    hidden_neurons = (h,)
    if hidden == None:
        print("\n----------- Neurons={0} -----------".format(hidden_neurons[0]))
        clf = MLPClassifier(solver='adam', hidden_layer_sizes=(hidden_neurons), max_iter = h)
    else:
        print("\n----------- Neurons={0} -----------".format(hidden[0]))
        clf = MLPClassifier(solver='adam', hidden_layer_sizes=(hidden), max_iter = iterations)
    clf.fit(X, y)
    return clf 

def evaluate_NN(clf, test_X, train_X, y, answer_token_map):
    answer_list = sorted(list(answer_token_map.keys()))
    print("----------- Iterations={0} -----------".format(clf.n_iter_))
    # TESTING SET
    predicted = one_hot_to_number(clf.predict(test_X))
    score = accuracy_score(y, predicted)
    report = classification_report(y, predicted, target_names=answer_list)
    print("Testing Set Score: {0}".format(score))
    print("Testing Report")
    print(report)
    # TRAINING SET
    predicted = one_hot_to_number(clf.predict(train_X))
    score = accuracy_score(y, predicted)
    report = classification_report(y, predicted, target_names=answer_list)
    print("Training Set Score: {0}".format(score))
    print("Training Report")
    print(report)

<a id='bAbI-Tasks'></a>
# bAbI-Tasks

Vamos a usar los siguientes parametros:

### SVM
* C_1 = 1
* C_2 = 3
* C_3 = 5
* C_4 = 7
* C_5 = 9

### NN
* Capa entrada = tamaño feature space
* Capa salida = tamaño vocabulario de respuestas
* Capa oculta = (0.7*feature_space_size+0.3*answers_vocab_size, 1)
* Iteraciones = tamaño de un set de capa oculta

<a id='bAbI-MAIN'></a>
## Main Function
[Volver al Menú](#Menu)

In [7]:
def intelligence_is_artificial(i):
    # Start info
    data_name = DATASET[i][0]
    OPTIONAL = DATASET[i][1]
    TRAIN_PATH = data_name+'_train.txt'
    TEST_PATH = data_name+'_test.txt'
    FILL_TOKEN = 'PAD'
    # Parse Data
    train_result = separate_story_question(TRAIN_PATH, OPTIONAL)
    train_stories = [element for element in train_result[0]]
    test_result = separate_story_question(TEST_PATH, OPTIONAL)
    test_stories = [element for element in test_result[0]]
    # Fill stories with tokens and create registers
    max_substory_length = max_story_lenght(train_stories+test_stories)
    train_stories = fill_with_fill_token(train_stories, max_substory_length, FILL_TOKEN)
    train_registers = create_register(train_result)
    test_stories = fill_with_fill_token(test_stories, max_substory_length, FILL_TOKEN)
    test_registers = create_register(test_result)
    # Dictionary of the vocabulary
    vocabulary_stories, vocabulary_answers = generate_vocabulary(train_registers, test_registers)
    # One_Hot for Training Data
    x_one_hot, y_one_hot = create_one_hot(train_registers, vocabulary_stories, vocabulary_answers)
    X = concatenate_one_hot_array(x_one_hot)
    y = answers_array(y_one_hot)
    # One_Hot for Testing Data
    test_x_one_hot, test_y_one_hot = create_one_hot(test_registers, vocabulary_stories, vocabulary_answers)
    test_X = concatenate_one_hot_array(test_x_one_hot)
    test_y = answers_array(test_y_one_hot)
    # sanitize one_hot answers
    y_one_hot = sanitize_one_hot(y_one_hot)
    test_y_one_hot = sanitize_one_hot(test_y_one_hot)
    # Information
    feature_space_size = len(X[0])
    stories_vocab_size = len(vocabulary_stories)
    answers_vocab_size = len(vocabulary_answers)
    max_substory_length = max_story_lenght(train_stories+test_stories)
    print("Tamaño del vocabulario de historias (n): %d" % stories_vocab_size)
    print("Tamaño del vocabulario de respuestas: %d" % answers_vocab_size)
    print("Largo máximo de subhistorias (m): %d" % max_substory_length)
    print("n(m + 1) = %d" % (stories_vocab_size * ( max_substory_length + 1)))
    print("Tamaño del feature space: %d" % feature_space_size)
    print("Vocabulario respuestas:")
    print(vocabulary_answers)
    # SVN
    print("\n")
    print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
    print("-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-")
    print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
    print("\n")
    SVC_1 = train_SVC(X, y, 1)
    SVC_2 = train_SVC(X, y, 3)
    SVC_3 = train_SVC(X, y, 5)
    SVC_4 = train_SVC(X, y, 7)
    SVC_5 = train_SVC(X, y, 9)
    evaluate_SVC(SVC_1, test_X, test_y, X, y, vocabulary_answers)
    evaluate_SVC(SVC_2, test_X, test_y, X, y, vocabulary_answers)
    evaluate_SVC(SVC_3, test_X, test_y, X, y, vocabulary_answers)
    evaluate_SVC(SVC_4, test_X, test_y, X, y, vocabulary_answers)
    evaluate_SVC(SVC_5, test_X, test_y, X, y, vocabulary_answers)
    # NN
    print("\n")
    print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
    print("-/-/-/-/-/-/-/-/-/- NEURAL NETWORK -/-/-/-/-/-/-/-/-/-/-/-/-/-")
    print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
    print("\n")
    NN = train_NN(X, y_one_hot)
    evaluate_NN(NN, test_X, X, y, vocabulary_answers)

<a id='bAbI-1'></a>
## 1 Basic factoid QA with single supporting fact
[Volver al Menú](#Menu)

In [17]:
intelligence_is_artificial(0)

Tamaño del vocabulario de historias (n): 17
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 14
n(m + 1) = 255
Tamaño del feature space: 255
Vocabulario respuestas:
{'bathroom': 0, 'bedroom': 1, 'garden': 2, 'hallway': 3, 'kitchen': 4, 'offic': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.434
Testing Report
             precision    recall  f1-score   support

   bathroom       0.36      0.38      0.37       149
    bedroom       0.46      0.33      0.38       171
     garden       0.47      0.51      0.49       187
    hallway       0.40      0.43      0.42       154
    kitchen       0.46      0.52      0.49       157
      offic       0.45      0.43      0.44       182

avg / total       0.44      0.43      0.43      1000

Training Set Score: 0.721
Training Repor

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (180) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

En este caso, no se produce overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

El resultado de set de entrenamiento puede ser overfitting

<a id='bAbI-2'></a>
## 2 Factoid QA with two supporting facts
[Volver al Menú](#Menu)

In [18]:
intelligence_is_artificial(1)

Tamaño del vocabulario de historias (n): 28
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 92
n(m + 1) = 2604
Tamaño del feature space: 2604
Vocabulario respuestas:
{'bathroom': 0, 'bedroom': 1, 'garden': 2, 'hallway': 3, 'kitchen': 4, 'offic': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.237
Testing Report
             precision    recall  f1-score   support

   bathroom       0.25      0.23      0.24       175
    bedroom       0.22      0.25      0.23       167
     garden       0.25      0.27      0.26       187
    hallway       0.26      0.19      0.22       160
    kitchen       0.24      0.19      0.21       165
      offic       0.21      0.29      0.24       146

avg / total       0.24      0.24      0.24      1000

Training Set Score: 1.0
Training Repor

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Hay un claro overfitting con el set en entrenamiento.

<a id='#bAbI-NN-1'></a>
### Análisis NN

Sí hay overfitting

<a id='bAbI-3'></a>
## 3 Factoid QA with three supporting facts
[Volver al Menú](#Menu)

In [19]:
intelligence_is_artificial(2)

Tamaño del vocabulario de historias (n): 28
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 232
n(m + 1) = 6524
Tamaño del feature space: 6524
Vocabulario respuestas:
{'bathroom': 0, 'bedroom': 1, 'garden': 2, 'hallway': 3, 'kitchen': 4, 'offic': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.209
Testing Report
             precision    recall  f1-score   support

   bathroom       0.26      0.29      0.27       185
    bedroom       0.18      0.19      0.19       154
     garden       0.21      0.13      0.16       215
    hallway       0.24      0.30      0.26       167
    kitchen       0.16      0.16      0.16       146
      offic       0.18      0.18      0.18       133

avg / total       0.21      0.21      0.21      1000

Training Set Score: 1.0
Training Repo

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Si hay overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si hay overfitting

<a id='bAbI-4'></a>
## 4 Two argument relations: subject vs. object
[Volver al Menú](#Menu)

In [20]:
intelligence_is_artificial(3)

Tamaño del vocabulario de historias (n): 11
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 2
n(m + 1) = 33
Tamaño del feature space: 33
Vocabulario respuestas:
{'bathroom': 0, 'bedroom': 1, 'garden': 2, 'hallway': 3, 'kitchen': 4, 'offic': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.679
Testing Report
             precision    recall  f1-score   support

   bathroom       0.71      0.61      0.66       177
    bedroom       0.69      0.66      0.67       156
     garden       0.66      0.77      0.71       167
    hallway       0.67      0.64      0.65       176
    kitchen       0.66      0.79      0.72       171
      offic       0.71      0.60      0.65       153

avg / total       0.68      0.68      0.68      1000

Training Set Score: 0.754
Training Report
 

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (24) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

No hay overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

No hay overfitting

<a id='bAbI-5'></a>
## 5  Three argument relations
[Volver al Menú](#Menu)

In [21]:
intelligence_is_artificial(4)

Tamaño del vocabulario de historias (n): 33
Tamaño del vocabulario de respuestas: 7
Largo máximo de subhistorias (m): 106
n(m + 1) = 3531
Tamaño del feature space: 3531
Vocabulario respuestas:
{'appl': 0, 'bill': 1, 'footbal': 2, 'fred': 3, 'jeff': 4, 'mari': 5, 'milk': 6}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.421
Testing Report
             precision    recall  f1-score   support

       appl       0.65      0.53      0.58       137
       bill       0.29      0.26      0.27       164
    footbal       0.59      0.59      0.59        95
       fred       0.41      0.45      0.43       188
       jeff       0.32      0.43      0.36       129
       mari       0.34      0.37      0.35       189
       milk       0.68      0.43      0.53        98

avg / total       0.44      0.42      0.

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Si hay overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

No hay overfitting

<a id='bAbI-6'></a>
## 6 Yes/No questions
[Volver al Menú](#Menu)

In [22]:
intelligence_is_artificial(5)

Tamaño del vocabulario de historias (n): 35
Tamaño del vocabulario de respuestas: 2
Largo máximo de subhistorias (m): 30
n(m + 1) = 1085
Tamaño del feature space: 1085
Vocabulario respuestas:
{'no': 0, 'yes': 1}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.506
Testing Report
             precision    recall  f1-score   support

         no       0.51      0.44      0.47       503
        yes       0.50      0.57      0.53       497

avg / total       0.51      0.51      0.50      1000

Training Set Score: 0.739
Training Report
             precision    recall  f1-score   support

         no       0.75      0.71      0.73       492
        yes       0.73      0.76      0.75       508

avg / total       0.74      0.74      0.74      1000


----------- C=3 -----------
Testing Set Score: 0.498
Te

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

No hay overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Sí hay overfitting

<a id='bAbI-7'></a>
## 7 Counting
[Volver al Menú](#Menu)

In [23]:
intelligence_is_artificial(6)

Tamaño del vocabulario de historias (n): 34
Tamaño del vocabulario de respuestas: 4
Largo máximo de subhistorias (m): 38
n(m + 1) = 1326
Tamaño del feature space: 1326
Vocabulario respuestas:
{'none': 0, 'one': 1, 'three': 2, 'two': 3}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.61
Testing Report
             precision    recall  f1-score   support

       none       0.64      0.56      0.60       468
        one       0.61      0.70      0.65       488
      three       0.00      0.00      0.00         2
        two       0.12      0.07      0.09        42

avg / total       0.60      0.61      0.60      1000

Training Set Score: 0.952
Training Report
             precision    recall  f1-score   support

       none       0.95      0.95      0.95       468
        one       0.95      0.95   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


----------- Iterations=145 -----------
Testing Set Score: 0.537
Testing Report
             precision    recall  f1-score   support

       none       0.53      0.59      0.55       468
        one       0.55      0.54      0.54       488
      three       0.00      0.00      0.00         1
        two       0.00      0.00      0.00        43

avg / total       0.51      0.54      0.52      1000

Training Set Score: 1.0
Training Report
             precision    recall  f1-score   support

       none       1.00      1.00      1.00       468
        one       1.00      1.00      1.00       488
      three       1.00      1.00      1.00         1
        two       1.00      1.00      1.00        43

avg / total       1.00      1.00      1.00      1000



<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Probable overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-8'></a>
## 8 Lists/Sets
[Volver al Menú](#Menu)

In [24]:
intelligence_is_artificial(7)

Tamaño del vocabulario de historias (n): 29
Tamaño del vocabulario de respuestas: 4
Largo máximo de subhistorias (m): 62
n(m + 1) = 1827
Tamaño del feature space: 1827
Vocabulario respuestas:
{'appl': 0, 'footbal': 1, 'milk': 2, 'noth': 3}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.482
Testing Report
             precision    recall  f1-score   support

       appl       0.47      0.44      0.45       226
    footbal       0.41      0.50      0.46       212
       milk       0.45      0.51      0.48       226
       noth       0.58      0.48      0.52       336

avg / total       0.49      0.48      0.48      1000

Training Set Score: 0.982
Training Report
             precision    recall  f1-score   support

       appl       0.98      0.98      0.98       231
    footbal       0.97      0.

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Si overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-9'></a>
## 9 Simple Negation
[Volver al Menú](#Menu)

In [25]:
intelligence_is_artificial(8)

Tamaño del vocabulario de historias (n): 24
Tamaño del vocabulario de respuestas: 2
Largo máximo de subhistorias (m): 14
n(m + 1) = 360
Tamaño del feature space: 360
Vocabulario respuestas:
{'no': 0, 'yes': 1}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.583
Testing Report
             precision    recall  f1-score   support

         no       0.66      0.73      0.69       638
        yes       0.41      0.33      0.36       362

avg / total       0.57      0.58      0.57      1000

Training Set Score: 0.731
Training Report
             precision    recall  f1-score   support

         no       0.75      0.85      0.80       633
        yes       0.67      0.52      0.59       367

avg / total       0.72      0.73      0.72      1000


----------- C=3 -----------
Testing Set Score: 0.58
Testi

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

No overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-10'></a>
## 10 Indefinite Knowledge
[Volver al Menú](#Menu)

In [26]:
intelligence_is_artificial(9)

Tamaño del vocabulario de historias (n): 23
Tamaño del vocabulario de respuestas: 3
Largo máximo de subhistorias (m): 14
n(m + 1) = 345
Tamaño del feature space: 345
Vocabulario respuestas:
{'maybe': 0, 'no': 1, 'yes': 2}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.445
Testing Report
             precision    recall  f1-score   support

      maybe       0.25      0.18      0.21       150
         no       0.46      0.49      0.48       437
        yes       0.47      0.49      0.48       413

avg / total       0.44      0.45      0.44      1000

Training Set Score: 0.674
Training Report
             precision    recall  f1-score   support

      maybe       0.78      0.57      0.65       122
         no       0.67      0.70      0.68       466
        yes       0.66      0.68      0.67      

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (242) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

No overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-11'></a>
## 11 Basic Coreference
[Volver al Menú](#Menu)

In [27]:
intelligence_is_artificial(10)

Tamaño del vocabulario de historias (n): 19
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 14
n(m + 1) = 285
Tamaño del feature space: 285
Vocabulario respuestas:
{'bathroom': 0, 'bedroom': 1, 'garden': 2, 'hallway': 3, 'kitchen': 4, 'offic': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.623
Testing Report
             precision    recall  f1-score   support

   bathroom       0.56      0.65      0.60       153
    bedroom       0.66      0.65      0.66       171
     garden       0.61      0.66      0.63       164
    hallway       0.69      0.59      0.63       187
    kitchen       0.62      0.59      0.61       170
      offic       0.61      0.59      0.60       155

avg / total       0.63      0.62      0.62      1000

Training Set Score: 0.856
Training Repor

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (201) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

No overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-12'></a>
## 12 Conjuction
[Volver al Menú](#Menu)

In [28]:
intelligence_is_artificial(11)

Tamaño del vocabulario de historias (n): 17
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 14
n(m + 1) = 255
Tamaño del feature space: 255
Vocabulario respuestas:
{'bathroom': 0, 'bedroom': 1, 'garden': 2, 'hallway': 3, 'kitchen': 4, 'offic': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.558
Testing Report
             precision    recall  f1-score   support

   bathroom       0.56      0.51      0.53       179
    bedroom       0.57      0.64      0.60       163
     garden       0.54      0.60      0.57       152
    hallway       0.58      0.49      0.53       189
    kitchen       0.55      0.50      0.52       169
      offic       0.56      0.63      0.59       148

avg / total       0.56      0.56      0.56      1000

Training Set Score: 0.82
Training Report

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (180) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

No overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Probable overfitting

<a id='bAbI-13'></a>
## 13 Compound Coreference
[Volver al Menú](#Menu)

In [29]:
intelligence_is_artificial(12)

Tamaño del vocabulario de historias (n): 19
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 14
n(m + 1) = 285
Tamaño del feature space: 285
Vocabulario respuestas:
{'bathroom': 0, 'bedroom': 1, 'garden': 2, 'hallway': 3, 'kitchen': 4, 'offic': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.809
Testing Report
             precision    recall  f1-score   support

   bathroom       0.79      0.81      0.80       172
    bedroom       0.83      0.74      0.79       183
     garden       0.81      0.84      0.82       179
    hallway       0.82      0.88      0.85       154
    kitchen       0.82      0.80      0.81       153
      offic       0.78      0.80      0.79       159

avg / total       0.81      0.81      0.81      1000

Training Set Score: 0.971
Training Repor

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (201) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Puede que sea overfitting, pero anduvo muy bien este

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-14'></a>
## 14 Time Reasoning
[Volver al Menú](#Menu)

In [30]:
intelligence_is_artificial(13)

Tamaño del vocabulario de historias (n): 21
Tamaño del vocabulario de respuestas: 6
Largo máximo de subhistorias (m): 18
n(m + 1) = 399
Tamaño del feature space: 399
Vocabulario respuestas:
{'bedroom': 0, 'cinema': 1, 'kitchen': 2, 'offic': 3, 'park': 4, 'school': 5}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.369
Testing Report
             precision    recall  f1-score   support

    bedroom       0.33      0.28      0.30       162
     cinema       0.29      0.34      0.31       171
    kitchen       0.44      0.38      0.41       176
      offic       0.42      0.38      0.40       177
       park       0.38      0.40      0.39       152
     school       0.37      0.43      0.40       162

avg / total       0.37      0.37      0.37      1000

Training Set Score: 0.886
Training Report
   

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (281) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Probable overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Sí overfitting

<a id='bAbI-15'></a>
## 15 Basic Deduction
[Volver al Menú](#Menu)

In [31]:
intelligence_is_artificial(14)

Tamaño del vocabulario de historias (n): 13
Tamaño del vocabulario de respuestas: 4
Largo máximo de subhistorias (m): 11
n(m + 1) = 156
Tamaño del feature space: 156
Vocabulario respuestas:
{'cat': 0, 'mous': 1, 'sheep': 2, 'wolf': 3}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.466
Testing Report
             precision    recall  f1-score   support

        cat       0.34      0.48      0.40       213
       mous       0.50      0.49      0.49       239
      sheep       0.40      0.41      0.40       207
       wolf       0.63      0.48      0.54       341

avg / total       0.49      0.47      0.47      1000

Training Set Score: 0.63
Training Report
             precision    recall  f1-score   support

        cat       0.64      0.68      0.66       290
       mous       0.61      0.63    

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (110) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Anduvo muy bien, buenos resultados

<a id='#bAbI-NN-1'></a>
### Análisis NN

Muy buenos resultados

<a id='bAbI-16'></a>
## 16 Basic Induction
[Volver al Menú](#Menu)

In [32]:
intelligence_is_artificial(15)

Tamaño del vocabulario de historias (n): 15
Tamaño del vocabulario de respuestas: 4
Largo máximo de subhistorias (m): 9
n(m + 1) = 150
Tamaño del feature space: 150
Vocabulario respuestas:
{'gray': 0, 'green': 1, 'white': 2, 'yellow': 3}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.511
Testing Report
             precision    recall  f1-score   support

       gray       0.51      0.47      0.49       251
      green       0.54      0.49      0.52       257
      white       0.53      0.52      0.52       256
     yellow       0.47      0.56      0.51       236

avg / total       0.51      0.51      0.51      1000

Training Set Score: 0.576
Training Report
             precision    recall  f1-score   support

       gray       0.56      0.53      0.55       226
      green       0.59      0.55

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (106) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

MUy buenos resultados

<a id='#bAbI-NN-1'></a>
### Análisis NN

Overfitting

<a id='bAbI-17'></a>
## 17 Positional Reasoning
[Volver al Menú](#Menu)

In [33]:
intelligence_is_artificial(16)

Tamaño del vocabulario de historias (n): 18
Tamaño del vocabulario de respuestas: 2
Largo máximo de subhistorias (m): 9
n(m + 1) = 180
Tamaño del feature space: 180
Vocabulario respuestas:
{'no': 0, 'yes': 1}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.495
Testing Report
             precision    recall  f1-score   support

         no       0.47      0.49      0.48       480
        yes       0.51      0.50      0.51       520

avg / total       0.50      0.49      0.50      1000

Training Set Score: 0.631
Training Report
             precision    recall  f1-score   support

         no       0.64      0.59      0.61       495
        yes       0.62      0.68      0.65       505

avg / total       0.63      0.63      0.63      1000


----------- C=3 -----------
Testing Set Score: 0.482
Testi

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (126) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Buenos resultados, no overitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-18'></a>
## 18 Size Reasoning
[Volver al Menú](#Menu)

In [34]:
intelligence_is_artificial(17)

Tamaño del vocabulario de historias (n): 18
Tamaño del vocabulario de respuestas: 2
Largo máximo de subhistorias (m): 23
n(m + 1) = 432
Tamaño del feature space: 432
Vocabulario respuestas:
{'no': 0, 'yes': 1}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.527
Testing Report
             precision    recall  f1-score   support

         no       0.55      0.56      0.56       531
        yes       0.50      0.49      0.49       469

avg / total       0.53      0.53      0.53      1000

Training Set Score: 0.665
Training Report
             precision    recall  f1-score   support

         no       0.67      0.64      0.65       495
        yes       0.66      0.69      0.68       505

avg / total       0.67      0.67      0.66      1000


----------- C=3 -----------
Testing Set Score: 0.531
Test

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Buenos resultados

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-19'></a>
## 19 Path Finding
[Volver al Menú](#Menu)

In [9]:
intelligence_is_artificial(18)

Tamaño del vocabulario de historias (n): 20
Tamaño del vocabulario de respuestas: 4
Largo máximo de subhistorias (m): 5
n(m + 1) = 120
Tamaño del feature space: 120
Vocabulario respuestas:
{'e': 0, 'n': 1, 's': 2, 'w': 3}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.272
Testing Report
             precision    recall  f1-score   support

          e       0.30      0.42      0.35       248
          n       0.25      0.21      0.23       269
          s       0.28      0.32      0.29       247
          w       0.23      0.14      0.17       236

avg / total       0.26      0.27      0.26      1000

Training Set Score: 0.366
Training Report
             precision    recall  f1-score   support

          e       0.35      0.43      0.39       268
          n       0.36      0.36      0.36      

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (85) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Underfitting probablemente

<a id='#bAbI-NN-1'></a>
### Análisis NN

Underfitting tambien

<a id='bAbI-20'></a>
## 20 Agents Motivation
[Volver al Menú](#Menu)

In [10]:
intelligence_is_artificial(19)

Tamaño del vocabulario de historias (n): 28
Tamaño del vocabulario de respuestas: 7
Largo máximo de subhistorias (m): 23
n(m + 1) = 672
Tamaño del feature space: 672
Vocabulario respuestas:
{'bedroom': 0, 'bore': 1, 'garden': 2, 'hungri': 3, 'kitchen': 4, 'thirsti': 5, 'tire': 6}


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------
Testing Set Score: 0.844
Testing Report
             precision    recall  f1-score   support

    bedroom       0.77      0.73      0.75        97
       bore       0.99      0.93      0.96       159
     garden       0.64      0.81      0.71        94
     hungri       0.84      0.72      0.78       157
    kitchen       0.84      0.84      0.84       180
    thirsti       0.78      0.87      0.82       152
       tire       0.99      0.94      0.97       161

avg / total       0.85      0.84 

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Si overfitting

<a id='#bAbI-NN-1'></a>
### Análisis NN

Si overfitting

<a id='bAbI-ALL'></a>
## All Together
[Volver al Menú](#Menu)

In [8]:
# Start info
data_name = DATASET[20][0]
OPTIONAL = DATASET[20][1]
TRAIN_PATH = 'dataset/en/'+data_name+'_train.txt'
TEST_PATH = 'dataset/en/'+data_name+'_test.txt'
FILL_TOKEN = 'PAD'
# Parse Data
train_result = separate_story_question(TRAIN_PATH, OPTIONAL)
train_stories = [element for element in train_result[0]]
test_result = separate_story_question(TEST_PATH, OPTIONAL)
test_stories = [element for element in test_result[0]]
# Fill stories with tokens and create registers
max_substory_length = max_story_lenght(train_stories+test_stories)
train_stories = fill_with_fill_token(train_stories, max_substory_length, FILL_TOKEN)
train_registers = create_register(train_result)
test_stories = fill_with_fill_token(test_stories, max_substory_length, FILL_TOKEN)
test_registers = create_register(test_result)
# Dictionary of the vocabulary
vocabulary_stories, vocabulary_answers = generate_vocabulary(train_registers, test_registers)
# One_Hot for Training Data
x_one_hot, y_one_hot = create_one_hot(train_registers, vocabulary_stories, vocabulary_answers)
X = concatenate_one_hot_array(x_one_hot)
y = answers_array(y_one_hot)
# One_Hot for Testing Data
test_x_one_hot, test_y_one_hot = create_one_hot(test_registers, vocabulary_stories, vocabulary_answers)
test_X = concatenate_one_hot_array(test_x_one_hot)
test_y = answers_array(test_y_one_hot)
# sanitize one_hot answers
y_one_hot = sanitize_one_hot(y_one_hot)
test_y_one_hot = sanitize_one_hot(test_y_one_hot)
# Information
feature_space_size = len(X[0])
stories_vocab_size = len(vocabulary_stories)
answers_vocab_size = len(vocabulary_answers)
max_substory_length = max_story_lenght(train_stories+test_stories)
print("Tamaño del vocabulario de historias (n): %d" % stories_vocab_size)
print("Tamaño del vocabulario de respuestas: %d" % answers_vocab_size)
print("Largo máximo de subhistorias (m): %d" % max_substory_length)
print("n(m + 1) = %d" % (stories_vocab_size * ( max_substory_length + 1)))
print("Tamaño del feature space: %d" % feature_space_size)
print("Vocabulario respuestas:")
print(vocabulary_answers)

Tamaño del vocabulario de historias (n): 144
Tamaño del vocabulario de respuestas: 40
Largo máximo de subhistorias (m): 232
n(m + 1) = 33552
Tamaño del feature space: 33552
Vocabulario respuestas:
{'appl': 0, 'bathroom': 1, 'bedroom': 2, 'bill': 3, 'bore': 4, 'cat': 5, 'cinema': 6, 'e': 7, 'footbal': 8, 'fred': 9, 'garden': 10, 'gray': 11, 'green': 12, 'hallway': 13, 'hungri': 14, 'jeff': 15, 'kitchen': 16, 'mari': 17, 'mayb': 18, 'milk': 19, 'mous': 20, 'n': 21, 'no': 22, 'none': 23, 'noth': 24, 'offic': 25, 'one': 26, 'park': 27, 's': 28, 'school': 29, 'sheep': 30, 'thirsti': 31, 'three': 32, 'tire': 33, 'two': 34, 'w': 35, 'white': 36, 'wolf': 37, 'ye': 38, 'yellow': 39}


In [9]:
print("\n")
print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
print("-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-")
print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
print("\n")
SVC_1 = train_SVC(X, y, 1)
#SVC_2 = train_SVC(X, y, 3)
#SVC_3 = train_SVC(X, y, 5)
#SVC_4 = train_SVC(X, y, 7)
#SVC_5 = train_SVC(X, y, 9)
evaluate_SVC(SVC_1, test_X, test_y, X, y, vocabulary_answers)
#evaluate_SVC(SVC_2, test_X, test_y, X, y, vocabulary_answers)
#evaluate_SVC(SVC_3, test_X, test_y, X, y, vocabulary_answers)
#evaluate_SVC(SVC_4, test_X, test_y, X, y, vocabulary_answers)
#evaluate_SVC(SVC_5, test_X, test_y, X, y, vocabulary_answers)



-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- SUPPORT VECTOR MACHINE -/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- C=1 -----------


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Testing Set Score: 0.50385
Testing Report
             precision    recall  f1-score   support

       appl       0.53      0.47      0.50       363
   bathroom       0.49      0.52      0.50      1190
    bedroom       0.49      0.54      0.51      1424
       bill       0.31      0.26      0.28       164
       bore       0.99      0.96      0.97       159
        cat       0.37      0.41      0.39       213
     cinema       0.32      0.35      0.33       171
          e       0.30      0.40      0.34       248
    footbal       0.48      0.55      0.51       307
       fred       0.41      0.38      0.39       188
     garden       0.52      0.54      0.53      1345
       gray       0.52      0.42      0.47       251
      green       0.57      0.39      0.46       257
    hallway       0.51      0.53      0.52      1187
     hungri       0.80      0.76      0.78       157
       jeff       0.28      0.34      0.31       129
    kitchen       0.56      0.50      0.53      1487
   

In [9]:
print("\n")
print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
print("-/-/-/-/-/-/-/-/-/- NEURAL NETWORK -/-/-/-/-/-/-/-/-/-/-/-/-/-")
print("-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-")
print("\n")
NN = train_NN(X, y_one_hot,(100,1), 10)
evaluate_NN(NN, test_X, X, y, vocabulary_answers)



-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/- NEURAL NETWORK -/-/-/-/-/-/-/-/-/-/-/-/-/-
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-



----------- Neurons=100 -----------


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


----------- Iterations=10 -----------


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Testing Set Score: 0.01835
Testing Report
             precision    recall  f1-score   support

       appl       0.02      1.00      0.04       367
   bathroom       0.00      0.00      0.00      1130
    bedroom       0.00      0.00      0.00      1384
       bill       0.00      0.00      0.00       151
       bore       0.00      0.00      0.00       164
        cat       0.00      0.00      0.00       290
     cinema       0.00      0.00      0.00       159
          e       0.00      0.00      0.00       268
    footbal       0.00      0.00      0.00       345
       fred       0.00      0.00      0.00       167
     garden       0.00      0.00      0.00      1305
       gray       0.00      0.00      0.00       226
      green       0.00      0.00      0.00       247
    hallway       0.00      0.00      0.00      1221
     hungri       0.00      0.00      0.00       151
       jeff       0.00      0.00      0.00       144
    kitchen       0.00      0.00      0.00      1486
   

<a id='#bAbI-SVN-1'></a>
### Análisis SVN

Usando todos los datos, se aprecia que el modelo alcanza niveles mucho más razonables y veridicos. El problema es el tiempo de ejecución. Mejor que usar cada test individual.

<a id='#bAbI-SVN-1'></a>
### Análisis NN

Entrenar este modelo es una pesadilla para mi computador. No pude correrlo con los parámetros normales que usaba en los otros sets de datos, por el tiempo de ejecución. Además, como no se puede usar GPU con scikit-learn, siento que no pude hacer una buena implementación de NN para este set.
Espero poder usar otra herramiento como Tensorflow y hacer un entrenamiento reforzado con capas ocultas de memoria, en ese caso yo creo que lograría un rendimiento notable.

<a id='Conclusión'></a>
# Conclusiones

[Volver al Menú](#Menu)


## Facilidad de uso
### SVM
Bastante sencillo de configurar
### NN
Muy entretenido configurar y pensar cómo modelar la red neuronal a usar

## Tiempo de proceso
### SVM
Bastante rápido, en comparación a NN
### NN
Muy díficil de procesar en un PC común y corriente, sobretodo el último set de datos. LA librería scikit-learn no es la adecuada para este tipo de trabajos por la falta deintegración con CUDA (GPU).

## Exactitud de clasificación
### SVM
En general fue más exacto, probablemente porque el problema es linealmente separable y se ajusta bien a este modelo.
### NN
Hay modelos más precisos que se ajustan a este tipo de problemas, por lo que podría mejorar muchisimo.

## Tipos de error más frecuentes
En general, para ambos modelos, el hecho de tener que pensar en mayor dimensionalidad dificulta el proceso de entendimiento y ejecución del problema.
### SVM
Entender dimensionalidades, problemas no linealmente separables funciona peor.
### NN
Cualquier error que se produzca luego de haber estado entrenando el modelo por una buena cantidad de tiempo, va a ser un dolor de cabeza y razón de desesperación para el pobre desarrollador. :(.